In [1]:
# If you run this notebook on Google Colaboratory, uncomment the below to install automl_alex.
#!pip install --quiet -U automl_alex

In [1]:
import automl_alex
import sklearn
import time
from automl_alex import AutoML, AutoMLClassifier
print(automl_alex.__version__)

0.07.24.2


In [2]:
RANDOM_SEED = 42

# Classifier

## Data

In [3]:
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
dataset = fetch_openml(name='credit-g', version=1, as_frame=True)
dataset.target = dataset.target.astype('category').cat.codes
dataset.data.head(5)

,checking_status,duration,credit_history,purpose,credit_amount,savings_status,employment,installment_commitment,personal_status,other_parties,residence_since,property_magnitude,age,other_payment_plans,housing,existing_credits,job,num_dependents,own_telephone,foreign_worker
0,<0,6.0,critical/other existing credit,radio/tv,1169.0,no known savings,>=7,4.0,male single,none,4.0,real estate,67.0,none,own,2.0,skilled,1.0,yes,yes
1,0<=X<200,48.0,existing paid,radio/tv,5951.0,<100,1<=X<4,2.0,female div/dep/mar,none,2.0,real estate,22.0,none,own,1.0,skilled,1.0,none,yes
2,no checking,12.0,critical/other existing credit,education,2096.0,<100,4<=X<7,2.0,male single,none,3.0,real estate,49.0,none,own,1.0,unskilled resident,2.0,none,yes
3,<0,42.0,existing paid,furniture/equipment,7882.0,<100,4<=X<7,2.0,male single,guarantor,4.0,life insurance,45.0,none,for free,1.0,skilled,2.0,none,yes
4,<0,24.0,delayed previously,new car,4870.0,<100,1<=X<4,3.0,male single,none,4.0,no known property,53.0,none,for free,2.0,skilled,2.0,none,yes


In [4]:
X_train, X_test, y_train, y_test = train_test_split(dataset.data, 
                                                    dataset.target,
                                                    test_size=0.25, 
                                                    random_state=RANDOM_SEED,)
X_train.shape, X_test.shape

((750, 20), (250, 20))

## Model

In [5]:
#model = AutoML(X_train, y_train, X_test, type_of_estimator='classifier', random_state=RANDOM_SEED)

# or Simply
model = AutoMLClassifier(X_train, y_train, X_test, random_state=RANDOM_SEED, verbose=1)

Source X_train shape:  (750, 20) | X_test shape:  (250, 20)
##################################################
Auto detect cat features:  13
> Start preprocessing Data
> Generate cat encodet features
 +  55  Features from  OneHotEncoder
 +  44  Features from  HelmertEncoder
 +  54  Features from  HashingEncoder
 +  16  Features from  FrequencyEncoder
> Generate Frequency Encode num features
 +  4  Frequency Encode Num Features 
> Clean Nans in num features
> Generate interaction Num Features
 +  24  Interaction Features
> Normalization Features
##################################################
> Total Features:  201
##################################################
New X_train shape:  (750, 201) | X_test shape:  (250, 201)


In [6]:
%%time
predict_test, predict_train = model.fit_predict(timeout=1000, verbose=2)

__________________________________________________
Step 1: Model 1
__________________________________________________
One iteration takes ~ 0.5 sec
> Start Auto calibration parameters
> Start optimization with the parameters:
CV_Folds =  10
Score_CV_Folds =  5
Feature_Selection =  True
Opt_lvl =  3
Cold_start =  59.0
Early_stoping =  118.0
Metric =  roc_auc_score
Direction =  maximize
##################################################
Default model OptScore = 0.7223
Optimize: : 106it [09:09,  5.19s/it,  | Model: CatBoost | OptScore: 0.792 | Best roc_auc_score: 0.8157 +- 0.023696]

 Predict from Models_1
 17%|█▋        | 1/6 [00:35<02:59, 35.81s/it]
 Mean Score roc_auc_score on 20 Folds: 0.7929 std: 0.033182
 33%|███▎      | 2/6 [01:19<02:32, 38.07s/it]
 Mean Score roc_auc_score on 20 Folds: 0.7903 std: 0.042993
 50%|█████     | 3/6 [01:25<01:25, 28.50s/it]
 Mean Score roc_auc_score on 20 Folds: 0.7904 std: 0.04597
 67%|██████▋   | 4/6 [01:59<01:00, 30.29s/it]
 Mean Score roc_auc_score 

In [7]:
predict_test[:5]

array([0.39651301, 0.40053729, 0.44835886, 0.37442077, 0.28559396])

In [8]:
print('Test AUC: ', round(sklearn.metrics.roc_auc_score(y_test, predict_test),4))

Test AUC:  0.7995


In [10]:
print('Test AUC: ', round(sklearn.metrics.roc_auc_score(y_test, predict_test),4))

Test AUC:  0.8079


In [9]:
model.stack_models_cfgs

,score_opt,model_score,score_std,model_name,model_param,wrapper_params,cat_encoders,columns,cv_folds
0,0.7920,0.8157,0.023696,CatBoost,"{'verbose': 0, 'early_stopping_rounds': 50, 't...",{'early_stopping': True},"[OneHotEncoder, HelmertEncoder, HashingEncoder...","(duration, credit_amount, OneHotEncoder_other_...",10
1,0.7796,0.8188,0.039219,CatBoost,"{'verbose': 0, 'early_stopping_rounds': 50, 't...",{'early_stopping': True},"[OneHotEncoder, HelmertEncoder, HashingEncoder...","(duration, credit_amount, OneHotEncoder_other_...",10
2,0.7787,0.8162,0.037515,CatBoost,"{'verbose': 0, 'early_stopping_rounds': 50, 't...",{'early_stopping': True},"[OneHotEncoder, HelmertEncoder, HashingEncoder...","(duration, credit_amount, OneHotEncoder_other_...",10
3,0.7781,0.8072,0.029129,CatBoost,"{'verbose': 0, 'early_stopping_rounds': 50, 't...",{'early_stopping': True},"[OneHotEncoder, HelmertEncoder, HashingEncoder...","(duration, credit_amount, OneHotEncoder_other_...",10
4,0.7776,0.8118,0.034186,CatBoost,"{'verbose': 0, 'early_stopping_rounds': 50, 't...",{'early_stopping': True},"[OneHotEncoder, HelmertEncoder, HashingEncoder...","(duration, credit_amount, OneHotEncoder_other_...",10
5,0.7771,0.8090,0.031916,CatBoost,"{'verbose': 0, 'early_stopping_rounds': 50, 't...",{'early_stopping': True},"[OneHotEncoder, HelmertEncoder, HashingEncoder...","(duration, credit_amount, OneHotEncoder_other_...",10
6,0.7606,0.7888,0.028190,KNeighbors,"{'n_jobs': -1, 'n_neighbors': 128, 'weights': ...",{},"[OneHotEncoder, HelmertEncoder, HashingEncoder...","(duration, credit_amount, OneHotEncoder_other_...",10
7,0.7595,0.7851,0.025573,KNeighbors,"{'n_jobs': -1, 'n_neighbors': 135, 'weights': ...",{},"[OneHotEncoder, HelmertEncoder, HashingEncoder...","(duration, credit_amount, OneHotEncoder_other_...",10
8,0.7588,0.7871,0.028315,KNeighbors,"{'n_jobs': -1, 'n_neighbors': 129, 'weights': ...",{},"[OneHotEncoder, HelmertEncoder, HashingEncoder...","(duration, credit_amount, OneHotEncoder_other_...",10
